# DAV 5400 Module 10 Assignment  
(Tidying and Reshaping Data)

**GitHub Repository URL** - https://raw.githubusercontent.com/tusharahuja01/DAV-5400/main/M10_Data.csv

## Reading Data from a CSV File on GitHub using pandas

To examine the data in the **'M10_Data.csv'** file, we first need to bring in the required libraries and load the data into a pandas DataFrame. Here's how we can begin:

In [123]:
import pandas as pd

# Loading the data in a DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/tusharahuja01/DAV-5400/main/M10_Data.csv')

# Displaying the first few rows of the data 
df.head()

,Month,Category,Caltex,Gulf,Mobil
0,Open,Engine Oil,140 : 000,199 : 000,141 : 000
1,NaN,GearBox Oil,198 : 000,132 : 000,121 : 000
2,Jan,Engine Oil,170 : 103,194 : 132,109 : 127
3,NaN,GearBox Oil,132 : 106,125 : 105,191 : 100
4,Feb,Engine Oil,112 : 133,138 : 113,171 : 101


After importing the necessary libraries, the **'M10_Data.csv'** file will be loaded into a pandas DataFrame called 'df'. This will allow you to take a quick look at the initial rows of data to understand its structure.

Once the data is loaded, you can proceed with any required cleaning, transformation, or analysis steps based on your specific needs. Please let me know what you would like me to do with this data next.

# Task - 1.1
**Use your knowledge of combining and reshaping data in Pandas to tidy and transform/reshape
the data contained within the dataframe. To get started, think about how you would want the data to appear if it were converted to “long” format, e.g., how would you define a “single observation” for the data shown in the graphic?; **How many key values are associated with each data value?; How many columns should your long format structure contain based on the information provided in the graphic shown above?; What would the column headings for the long structure be?**; etc. Use your answers to these questions to guide your reshaping/transformational work on the data. Your reshaping/transformational steps must include converting the above table to a “tidy” long format. Additional transformational steps (e.g., filling in missing data values, renaming columns, etc.) should be performed as needed to ensure that your data is, in fact, “tidy”.**

## Answer:

To reshape the data into a tidy "long" format, we need to think about how we want to define a single observation, we need to identify the key variables and the value variables. In this case, the key variables are "Month", "Category", and "Supplier" (inferred from the "Caltex", "Gulf", and "Mobil" columns), while the value variable is the number representing the purchases and consumption (e.g., "purchased : consumed").

**Here's how we can reshape the data into a "long" format using pandas**

1. Month (or "Open" for the starting inventory)
2. Category (gearbox or engine)
3. Supplier (Caltex, Gulf, or Mobil)
4. Consumption (e.g., "purchased : consumed")

**Additionally, there are two data values associated with each observation: (Consumption)**

1. Purchased amount
2. Consumed amount

**Therefore, the long format structure should contain the following columns:**

1. Month
2. Category
3. Supplier
4. Purchased
5. Consumed

Here's how we can reshape the data into a long format and perform additional transformations to make it tidy:


In [124]:
# Check the shape of the data
print(df.shape)

(14, 5)


In [125]:
# Check the columns of the data
print(df.columns)

Index(['Month', 'Category', 'Caltex', 'Gulf', 'Mobil'], dtype='object')


In [126]:
# Converting the DataFrame to a stacked/melted representation.
melted = pd.melt(df, id_vars=['Month', 'Category'], value_vars=['Caltex', 'Gulf', 'Mobil'], var_name='Supplier', value_name='Value')

# Separating the 'Value' column into separate 'Purchased' and 'Consumed' columns
melted[['Purchased', 'Consumed']] = pd.DataFrame(melted['Value'].str.split(':').tolist(), dtype=float).values

# Removing the original 'Value' column from axis
melted = melted.drop('Value', axis=1)

# Handle missing values by propagating the last valid observation forward
data = melted.fillna(method='ffill')

# Showing the result
print(data)

   Month     Category Supplier  Purchased  Consumed
0   Open   Engine Oil   Caltex      140.0       0.0
1   Open  GearBox Oil   Caltex      198.0       0.0
2    Jan   Engine Oil   Caltex      170.0     103.0
3    Jan  GearBox Oil   Caltex      132.0     106.0
4    Feb   Engine Oil   Caltex      112.0     133.0
5    Feb  GearBox Oil   Caltex      193.0     148.0
6    Mar   Engine Oil   Caltex      184.0     100.0
7    Mar  GearBox Oil   Caltex      138.0     121.0
8    Apr   Engine Oil   Caltex      149.0     150.0
9    Apr  GearBox Oil   Caltex      185.0     125.0
10   May   Engine Oil   Caltex      170.0     139.0
11   May  GearBox Oil   Caltex      168.0     117.0
12   Jun   Engine Oil   Caltex      159.0     129.0
13   Jun  GearBox Oil   Caltex      107.0     129.0
14  Open   Engine Oil     Gulf      199.0       0.0
15  Open  GearBox Oil     Gulf      132.0       0.0
16   Jan   Engine Oil     Gulf      194.0     132.0
17   Jan  GearBox Oil     Gulf      125.0     105.0
18   Feb   E

**Here's what the code does:**

1. Read the CSV file from the GitHub URL and store it in the df DataFrame.
2. Use pd.melt to convert the DataFrame to long format. The id_vars are the columns that remain constant, and the value_vars are the columns that will be melted into rows. The var_name and value_name are the new column names for the melted data.
3. Split the 'Value' column into 'Purchased' and 'Consumed' using str.split and convert the strings to floats.
4. Drop the original 'Value' column as it is no longer needed.
5. Fill in any missing values using fillna in dataframe 'data'.
6. Print the transformed DataFrame.

**The resulting DataFrame will have the following columns:**

* Month
* Category
* Supplier
* Purchased
* Consumed

Each row represents a single observation with the key variables ("Month", "Category", "Supplier") and the value variables ("Purchased", "Consumed"). This is a tidy long format, where each column represents a distinct variable, and each row represents a single observation

# Task - 1.2

**Using your reshaped/transformed data, perform analysis to answer the following questions:**


## Answer:


### 1. What was the amount of oil remaining at the end of the chronological period for each pair of oil types/brands?

In [ ]:
# We will group the data by 'Category' and 'Supplier' 
grouping = melted.groupby(['Category', 'Supplier'])

This line groups the melted DataFrame based on the combination of 'Category' and 'Supplier' columns. It creates a DataFrameGroupBy object grouped where each group contains rows with the same 'Category' and 'Supplier' values.

In [ ]:
# Calculating the total amount left and taking the sum for each group
amount_left = grouped[['Purchased', 'Consumed']].sum().reset_index()
amount_left['Remaining_Amount'] = amount_left['Purchased'] - amount_left['Consumed']

* **grouped[['Purchased', 'Consumed']].sum():** This selects the 'Purchased' and 'Consumed' columns from the grouped data and calculates the sum of each column for every group.
* **.reset_index():** This method converts the resulting DataFrame from the previous step into a regular DataFrame with a new index (0, 1, 2, ...) and the original group labels ('Category' and 'Supplier') as separate columns.
* **amount_left['Remaining'] = amount_left['Purchased'] - amount_left['Consumed']:** This line creates a new column 'Remaining' in the amount_left DataFrame by subtracting the 'Consumed' column from the 'Purchased' column. This calculation gives the amount of oil remaining for each group.

The purpose of this step is to calculate the amount left of oil at the end of the chronological period for each combination of 'Category' and 'Supplier'.

In [ ]:
# Results Displayed
print("Amount of oil remaining at the end of the chronological period:")
print(amount_left[['Category', 'Supplier', 'Purchased', 'Consumed', 'Remaining_Amount']])

This part of the code prints a header message and displays the amount_left DataFrame with the 'Category', 'Supplier', 'Purchased', 'Consumed', and 'Remaining_Amount' columns. This output shows the amount of oil remaining at the end of the chronological period for each pair of oil types/brands, along with the total purchased and consumed amounts.


##  Final Code

In [127]:
# We will group the data by 'Category' and 'Supplier' 
grouping = melted.groupby(['Category', 'Supplier'])

# Taking the sum for each group
amount_left = grouping[['Purchased', 'Consumed']].sum().reset_index()

# Calculating the total amount left
# amount_left['Remaining_Amount'] = amount_left['Purchased'].sub(amount_left['Consumed'])
amount_left['Remaining_Amount'] = amount_left['Purchased'] - amount_left['Consumed']

# Results Displayed
print("Total Amount of oil remaining at the end of the chronological period is:\n")
print(amount_left[['Category', 'Supplier', 'Purchased', 'Consumed', 'Remaining_Amount']])

Total Amount of oil remaining at the end of the chronological period is:

      Category Supplier  Purchased  Consumed  Remaining_Amount
0   Engine Oil   Caltex     1084.0     754.0             330.0
1   Engine Oil     Gulf     1063.0     761.0             302.0
2   Engine Oil    Mobil     1021.0     676.0             345.0
3  GearBox Oil   Caltex     1121.0     746.0             375.0
4  GearBox Oil     Gulf     1152.0     733.0             419.0
5  GearBox Oil    Mobil     1020.0     721.0             299.0


The code performs the necessary data manipulation and calculations to answer the question "What was the amount of oil remaining at the end of the chronological period for each pair of oil types/brands?" by grouping the data, summing the 'Purchased' and 'Consumed' values, and calculating the difference to find the 'Remaining' amount.

### 2. At the end of the chronological period, what was the most consumed brand of oil for each of the two separate types of oil ?

In [ ]:
# Grouping the data by 'Category' and 'Supplier' columns, and doing sum of the 'Consumed' amounts
consumed_amounts = melted.groupby(['Category', 'Supplier'])['Consumed'].sum().reset_index()


* **melted.groupby(['Category', 'Supplier'])['Consumed']:** This line groups the melted DataFrame by the combination of 'Category' and 'Supplier' columns, and then selects the 'Consumed' column.
* **.sum():** This method calculates the sum of the 'Consumed' values for each group.
* **.reset_index():** This method converts the resulting Series from the previous step into a DataFrame with a new index (0, 1, 2, ...) and the original group labels ('Category' and 'Supplier') as separate columns.

The resulting consumed_amounts DataFrame contains the total 'Consumed' amount for each combination of 'Category' and 'Supplier'.

In [ ]:
# Sorting the data by 'Category' (ascending order) and 'Consumed' (descending order) to get the most consumed brand
most_consumed = consumed_amounts.sort_values(by=['Category']).sort_values(by=['Consumed'], ascending=False)

This line sorts the consumed_amounts DataFrame based on two criteria:

* 'Category' in ascending order (to group the oil types together)
* 'Consumed' in descending order (to place the most consumed brand at the top for each category)

The sorted DataFrame is stored in the most_consumed variable.

In [ ]:
# Displaying the result
print("\nThe Most consumed brand of oil for each type of supplier's are:\n")
print(most_consumed.groupby('Category').head(1)[['Category', 'Supplier', 'Consumed']])

* **most_consumed.groupby('Category'):** This line groups the most_consumed DataFrame by the 'Category' column.
* **.head(1):** This method selects the first row (the row with the largest 'Consumed' value) from each group.
* **[['Category', 'Supplier', 'Consumed']]:** This selects the 'Category', 'Supplier', and 'Consumed' columns from the resulting DataFrame.

The output shows the most consumed brand of oil for each type ('Category') by displaying the 'Category', 'Supplier', and 'Consumed' values for the brand with the highest consumption within each category.

## Final Code

In [128]:
# Grouping the data by 'Category' and 'Supplier' columns, and taking sum of the 'Consumed' amounts
consumed_amounts = melted.groupby(['Category', 'Supplier'])['Consumed'].sum().reset_index()

# Sorting the data by 'Category' (ascending order) and 'Consumed' (descending order) to get the most consumed brand
most_consumed = consumed_amounts.sort_values(by=['Category']).sort_values(by=['Consumed'], ascending=False)

# Displaying the result
print("The Most consumed brand of oil for each type of supplier's are:\n")
print(most_consumed.groupby('Category').head(1)[['Category', 'Supplier', 'Consumed']])

The Most consumed brand of oil for each type of supplier's are:

      Category Supplier  Consumed
1   Engine Oil     Gulf     761.0
3  GearBox Oil   Caltex     746.0


In summary, this code first calculates the total consumed amount for each combination of 'Category' and 'Supplier'. It then sorts the data by 'Category' and 'Consumed' (in descending order) to bring the most consumed brand for each category to the top. Finally, it displays the 'Category', 'Supplier', and 'Consumed' values for the most consumed brand within each category.

## Task - 1.3

**Finally, given your “tidy” long format structure, describe what, if any, changes you would make to the visual presentation of the data if you were then asked to transform your “long” data back into a “wide” format: would you mimic the structure of the graphic shown above? If not, how might you transform your “long” data to “wide” format to make its “wide” presentation easier to understand and work with? Provide an example of your recommendation and explain your rationale for preferring your specific structure**

When transforming the "long" data back into a "wide" format, it's essential to consider how the wide format will be used and interpreted. Mimicking the structure of the original graphic might not always be the best approach, especially if the original format has limitations in terms of readability or ease of analysis.

Here's a recommendation for transforming the "long" data into a "wide" format to improve its presentation and usability:

1. Instead of having separate columns for each supplier (e.g., 'Caltex', 'Gulf', 'Mobil'), we can have one column for the supplier and another column for the oil type (e.g., 'Engine' and 'Gearbox').
2. The values in the wide format will represent either the amount purchased or consumed for each supplier-oil type combination.

In [129]:
# Creating a pivot table so that we can transform the data into wide format
wide_format = data.pivot_table(index=['Month', 'Supplier'], columns='Category', values=['Purchased', 'Consumed'], aggfunc='sum')

# Flatten the column index (Multi-Level)
wide_format.columns = ['%s_%s' % (col[0], col[1]) for col in wide_format.columns.values]

# We will reset the index to make Month and Supplier as separate columns
wide_format.reset_index(inplace=True)

# Renaming the columns
wide_format.rename(columns={'Purchased_Engine': 'Purchased_Engine',
                             'Purchased_Gearbox': 'Purchased_Gearbox',
                             'Purchased_Gearbox Oil': 'Purchased_Gearbox_Oil',
                             'Consumed_Engine': 'Consumed_Engine',
                             'Consumed_Gearbox': 'Consumed_Gearbox',
                             'Consumed_Gearbox Oil': 'Consumed_Gearbox_Oil'},
                   inplace=True)


# Displaying the transformed wide_format data below
print(wide_format)


   Month Supplier  Consumed_Engine Oil  Consumed_GearBox Oil  \
0    Apr   Caltex                150.0                 125.0   
1    Apr     Gulf                118.0                 133.0   
2    Apr    Mobil                118.0                 121.0   
3    Feb   Caltex                133.0                 148.0   
4    Feb     Gulf                113.0                 119.0   
5    Feb    Mobil                101.0                 127.0   
6    Jan   Caltex                103.0                 106.0   
7    Jan     Gulf                132.0                 105.0   
8    Jan    Mobil                127.0                 100.0   
9    Jun   Caltex                129.0                 129.0   
10   Jun     Gulf                138.0                 141.0   
11   Jun    Mobil                105.0                 112.0   
12   Mar   Caltex                100.0                 121.0   
13   Mar     Gulf                141.0                 133.0   
14   Mar    Mobil                108.0  

**Rationale for preferring this structure:**

* Simplicity: The structure is simpler, with fewer columns compared to mimicking the original graphic. It's easier to read and understand.
* Flexibility: This structure allows for easier manipulation and analysis of the data. For example, it facilitates filtering, grouping, and aggregating based on either supplier or oil type.
* Consistency: The structure maintains consistency in terms of column names and data representation, making it easier to work with programmatically.
* Scalability: If there are additional suppliers or oil types in the future, this structure can easily accommodate them without the need for extensive modifications.

Overall, transforming the "long" data into this "wide" format enhances its readability, usability, and flexibility for analysis.

## Another Method

In [130]:
# Create a pivot table to transform the data into wide format
wide_format = melted.pivot_table(index=['Month', 'Supplier'], columns='Category', values=['Purchased', 'Consumed'], aggfunc='sum')

# Flatten the multi-level column index
wide_format.columns = [f'{col[1]}_{col[0]}' for col in wide_format.columns]


# Reset index to make Month and Supplier as columns
wide_format.reset_index(inplace=True)

# Display the transformed wide format data
print(wide_format)


   Month Supplier  Engine Oil_Consumed  Engine Oil_Purchased
0    Apr   Caltex                150.0                 149.0
1    Apr     Gulf                118.0                 117.0
2    Apr    Mobil                118.0                 117.0
3    Feb   Caltex                133.0                 112.0
4    Feb     Gulf                113.0                 138.0
5    Feb    Mobil                101.0                 171.0
6    Jan   Caltex                103.0                 170.0
7    Jan     Gulf                132.0                 194.0
8    Jan    Mobil                127.0                 109.0
9    Jun   Caltex                129.0                 159.0
10   Jun     Gulf                138.0                 170.0
11   Jun    Mobil                105.0                 169.0
12   Mar   Caltex                100.0                 184.0
13   Mar     Gulf                141.0                 141.0
14   Mar    Mobil                108.0                 114.0
15   May   Caltex       

**NOTE :**

The original code wide_format.columns = [f'{col[1]}_{col[0]}' for col in wide_format.columns] assumes that the multi-level column index has only two levels, where col[0] is the first level (e.g., 'Purchased' or 'Consumed'), and col[1] is the second level (e.g., 'Engine' or 'Gearbox'). However, when you have a third level ('Gearbox Oil'), this code will not work correctly.

The updated code wide_format.columns = ['_'.join(col).strip() for col in wide_format.columns.values] joins all the levels of the column index with an underscore (_) and removes any leading/trailing whitespace using strip(). This way, it will handle multi-level column indexes with any number of levels.

In [132]:
# Pivot the DataFrame to transform it back into a wide format
wide_format_data = data.pivot_table(index=['Month', 'Category'], columns=['Supplier', 'Category'], values=['Purchased', 'Consumed'])

# Flatten the multi-level columns

wide_format_data.columns = [' '.join(col).strip() for col in wide_format_data.columns.values]
# wide_format.columns = [col.replace(' ', '_') for col in wide_format.columns.values]

# wide_format.columns = [''.join(c for c in col if c != ' ') for col in wide_format.columns.values]

# Reset the index to make 'Month' and 'Category' as regular columns
wide_format_data.reset_index(inplace=True)

print(wide_format_data)

   Month     Category  Consumed Caltex Engine Oil  \
0    Apr   Engine Oil                       150.0   
1    Apr  GearBox Oil                         NaN   
2    Feb   Engine Oil                       133.0   
3    Feb  GearBox Oil                         NaN   
4    Jan   Engine Oil                       103.0   
5    Jan  GearBox Oil                         NaN   
6    Jun   Engine Oil                       129.0   
7    Jun  GearBox Oil                         NaN   
8    Mar   Engine Oil                       100.0   
9    Mar  GearBox Oil                         NaN   
10   May   Engine Oil                       139.0   
11   May  GearBox Oil                         NaN   
12  Open   Engine Oil                         0.0   
13  Open  GearBox Oil                         NaN   

    Consumed Caltex GearBox Oil  Consumed Gulf Engine Oil  \
0                           NaN                     118.0   
1                         125.0                       NaN   
2                    